In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from alpha_vantage.timeseries import TimeSeries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from ta.momentum import RSIIndicator
from ta.trend import MACD
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")


In [2]:
def fetch_data(symbol, api_key=API_KEY):
    ts = TimeSeries(key=api_key, output_format='pandas')
    data, _ = ts.get_daily(symbol=symbol, outputsize='full')
    data = data.sort_index()
    data = data.rename(columns={
        '1. open': 'open',
        '2. high': 'high',
        '3. low': 'low',
        '4. close': 'close',
        '5. volume': 'volume'
    })
    return data


In [ ]:
# Apply indicators per ticker
def add_features(df):
    df['return'] = df['close'].pct_change()
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    macd = MACD(df['close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['target'] = np.where(df['return'].shift(-1) > 0, 1, 0)
    df.dropna(inplace=True)
    return df


In [ ]:
# Train Random Forest model and calculate performance
def train_evaluate(df):
    features = ['rsi', 'macd', 'macd_signal']
    X = df[features]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
    
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    df = df.iloc[-len(y_pred):].copy()
    df['pred'] = y_pred
    df['strategy'] = np.where(df['pred'] == 1, df['return'], 0)
    
    sharpe = np.sqrt(252) * df['strategy'].mean() / df['strategy'].std()
    cumulative = (1 + df['strategy']).cumprod().iloc[-1]
    return sharpe, cumulative, df['strategy']


In [5]:
# Suppose we already have strategy_returns = { 'AAPL': series1, 'MSFT': series2, ... }

from scipy.optimize import minimize

def optimize_portfolio(returns_dict):
    returns_df = pd.DataFrame(returns_dict)
    weights = np.ones(len(returns_df.columns)) / len(returns_df.columns)

    def neg_sharpe(w):
        port_return = (returns_df * w).sum(axis=1)
        return -np.sqrt(252) * port_return.mean() / port_return.std()

    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1)] * len(weights)

    result = minimize(neg_sharpe, weights, bounds=bounds, constraints=constraints)
    return result.x, -result.fun


In [6]:
example_tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'JPM', 'BA']

print("🔢 Example Tickers:")
for i, ticker in enumerate(example_tickers):
    print(f"{i+1}. {ticker}")

chosen_indices = input("Please choose 5 ticker indices (comma-separated, e.g. 1,3,5,7,9): ")
chosen_tickers = [example_tickers[int(i)-1] for i in chosen_indices.split(',')]

print(f"\n✅ You selected: {chosen_tickers}")


🔢 Example Tickers:
1. AAPL
2. MSFT
3. GOOG
4. AMZN
5. META
6. TSLA
7. NFLX
8. NVDA
9. JPM
10. BA

✅ You selected: ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META']


In [8]:
import json



# ✅ Save to file for reuse
ticker_save_path = "/workspaces/backtesting/investment-portfolio-project/data/selected_tickers.json"
with open(ticker_save_path, 'w') as f:
    json.dump(chosen_tickers, f)

print(f"💾 Selected tickers saved to {ticker_save_path}")


💾 Selected tickers saved to /workspaces/backtesting/investment-portfolio-project/data/selected_tickers.json


In [9]:
# Load saved tickers
with open("/workspaces/backtesting/investment-portfolio-project/data/selected_tickers.json", 'r') as f:
    chosen_tickers = json.load(f)

print("📂 Using previously saved tickers:", chosen_tickers)


📂 Using previously saved tickers: ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META']


In [10]:
strategy_returns = {}
sharpe_scores = {}
cumulative_returns = {}

for ticker in chosen_tickers:
    print(f"\n📥 Processing {ticker}...")
    
    try:
        data = fetch_data(ticker)
        data = add_features(data)

        sharpe, cumulative, strat_series = train_evaluate(data)
        
        strategy_returns[ticker] = strat_series
        sharpe_scores[ticker] = sharpe
        cumulative_returns[ticker] = cumulative
        # Buy & Hold return series (baseline strategy)
        buyhold_returns = data['return'].loc[strat_series.index]  # align with strategy period
        strategy_returns[f"{ticker}_buyhold"] = buyhold_returns
        # RSI signal strategy
        data['rsi_signal'] = np.where(data['rsi'] < 30, 1, 0)
        data['rsi_signal'] = np.where(data['rsi'] > 70, -1, data['rsi_signal'])

        # RSI strategy returns
        rsi_returns = data['rsi_signal'].shift(1) * data['return']
        rsi_returns = rsi_returns.loc[strat_series.index]  # align dates

        strategy_returns[f"{ticker}_rsi"] = rsi_returns



        print(f"📊 {ticker} - Sharpe: {sharpe:.2f}, Cumulative Return: {cumulative:.2%}")
    
    except Exception as e:
        print(f"❌ Error processing {ticker}: {e}")



📥 Processing AAPL...
📊 AAPL - Sharpe: -0.09, Cumulative Return: 40.29%

📥 Processing MSFT...
📊 MSFT - Sharpe: 0.13, Cumulative Return: 103.33%

📥 Processing GOOG...
📊 GOOG - Sharpe: -0.09, Cumulative Return: 90.99%

📥 Processing AMZN...
📊 AMZN - Sharpe: -0.68, Cumulative Return: 1.92%

📥 Processing META...
📊 META - Sharpe: 1.07, Cumulative Return: 174.93%


In [11]:
# Ask user for weight between Sharpe and Return
try:
    alpha = float(input("Enter the weight for Sharpe Ratio (between 0 and 1): "))
    if not 0 <= alpha <= 1:
        raise ValueError
except:
    print("Invalid input. Using default alpha = 0.5")
    alpha = 0.5

print(f"⚖️ Optimization will use α = {alpha:.2f} → {alpha:.0%} Sharpe, {(1-alpha):.0%} Return")


⚖️ Optimization will use α = 0.50 → 50% Sharpe, 50% Return


In [12]:
from scipy.optimize import minimize
import numpy as np

def annual_return(returns):
    return (1 + returns).prod()**(252 / len(returns)) - 1

def sharpe_ratio(returns, risk_free_rate=0.0):
    excess = returns - risk_free_rate
    return np.sqrt(252) * excess.mean() / excess.std()

def combined_objective(weights, returns_matrix, alpha):
    portfolio_returns = returns_matrix.dot(weights)
    sharpe = sharpe_ratio(portfolio_returns)
    cagr = annual_return(portfolio_returns)
    return - (alpha * sharpe + (1 - alpha) * cagr)  # Negative to minimize

def optimize_combined_strategy(returns_matrix, alpha):
    n = returns_matrix.shape[1]
    init_guess = np.ones(n) / n
    bounds = [(0.0, 1.0)] * n
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

    result = minimize(combined_objective, init_guess,
                      args=(returns_matrix, alpha),
                      method='SLSQP',
                      bounds=bounds,
                      constraints=constraints)
    return result.x, -result.fun  # weights, score


In [14]:
# Separate by strategy type
rf_strategies = {k: v for k, v in strategy_returns.items() if not any(x in k.lower() for x in ['rsi', 'buyhold'])}
rsi_strategies = {k: v for k, v in strategy_returns.items() if 'rsi' in k.lower()}
buyhold_strategies = {k: v for k, v in strategy_returns.items() if 'buyhold' in k.lower()}

strategy_sets = {
    'Random Forest': rf_strategies,
    'RSI': rsi_strategies,
    'Buy & Hold': buyhold_strategies
}

optimized_weights = {}
cumulative_returns = {}

for name, strat_dict in strategy_sets.items():
    returns_df = pd.concat(strat_dict.values(), axis=1).fillna(0)
    weights, _ = optimize_combined_strategy(returns_df, alpha)
    portfolio_returns = returns_df.dot(weights)
    cumulative_returns[name] = (1 + portfolio_returns).cumprod()
    optimized_weights[name] = weights
    print(f"✅ Optimized {name} Portfolio — Final Value: {cumulative_returns[name].iloc[-1]:.4f}")


✅ Optimized Random Forest Portfolio — Final Value: 1.7493
✅ Optimized RSI Portfolio — Final Value: 1.5377
✅ Optimized Buy & Hold Portfolio — Final Value: 7.8072


In [20]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Cargar los datos
feature_data = pd.read_csv('/workspaces/backtesting/investment-portfolio-project/data/data_features.csv', parse_dates=['date'])
signal_data = pd.read_csv('/workspaces/backtesting/investment-portfolio-project/data/signal_data.csv', parse_dates=['date'])

# Usar 'date' como índice en ambos
feature_data.set_index('date', inplace=True)
signal_data.set_index('date', inplace=True)

# Calcular señales clásicas
feature_data['SMA_10'] = feature_data['close'].rolling(window=10).mean()
feature_data['SMA_50'] = feature_data['close'].rolling(window=50).mean()
feature_data['y_pred_crossover'] = np.where(feature_data['SMA_10'] > feature_data['SMA_50'], 1, 0)

feature_data['y_pred_rsi'] = np.where(feature_data['rsi'] < 30, 1, 0)
feature_data['y_pred_rsi'] = np.where(feature_data['rsi'] > 70, 0, feature_data['y_pred_rsi'])

# Seleccionar columnas de señal
strategy_signals = feature_data[['y_pred_crossover', 'y_pred_rsi']]

# Actualizar columnas en signal_data con las del nuevo cálculo
signal_data.update(strategy_signals)

# Guardar el resultado
output_path = '/workspaces/backtesting/investment-portfolio-project/data/signal_data.csv'
signal_data.to_csv(output_path)

print(f"✅ Estrategias clásicas actualizadas y archivo guardado en: {output_path}")



✅ Estrategias clásicas actualizadas y archivo guardado en: /workspaces/backtesting/investment-portfolio-project/data/signal_data.csv


In [22]:
print("\n💼 Optimized Portfolio Weights (Random Forest):")
for ticker, weight in zip(rf_strategies.keys(), weights):
    print(f"{ticker}: {weight:.2%}")
print(f"\n🚀 Optimized Portfolio Sharpe Ratio (RF): {sharpe:.2f}")



💼 Optimized Portfolio Weights (Random Forest):
AAPL: 0.00%
MSFT: 0.00%
GOOG: 0.00%
AMZN: 0.00%
META: 100.00%

🚀 Optimized Portfolio Sharpe Ratio (RF): 1.07


In [23]:
import joblib

weights_path = "/workspaces/backtesting/investment-portfolio-project/models/optimized_weights.pkl"
joblib.dump(optimized_weights, weights_path)
print(f"💾 Optimized weights saved to: {weights_path}")


💾 Optimized weights saved to: /workspaces/backtesting/investment-portfolio-project/models/optimized_weights.pkl


In [24]:
cumulative_returns = {}
for label, (weights, strategy_dict) in portfolios.items():
    matrix = pd.concat(strategy_dict.values(), axis=1).fillna(0)
    weighted_returns = matrix.dot(weights)
    cumulative_returns[label] = (1 + weighted_returns).cumprod()
pd.DataFrame(cumulative_returns).plot(figsize=(12,6), title="Cumulative Returns: RF vs RSI vs Buy & Hold")
plt.ylabel("Cumulative Return")
plt.grid(True)
plt.show()


NameError: name 'portfolios' is not defined